In [1]:

import tganalysis
import os
import emoji
import pandas as pd
import numpy as np


paths = ['C:/Users/mk/Downloads/Telegram Desktop/ChatExport_2024-04-20 (2)']

filename = os.path.join('data', paths[0], 'result.json')


In [2]:

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
emb = NewsEmbedding()
segmenter = Segmenter()
morph_vocab = MorphVocab()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)


In [3]:
tg = tganalysis.tg_json_analysis(filename)
data = tg.df
data

{66396: msg(id=66396, type='message', date='2023-10-01T00:08:47', from_name='Aura_ Pirs', from_id='user1206893398', reply_to_message_id=66395, text='Уже объяснили и это печально', words=None, year=2023, month=10, day=1, week=39, weekday=7, hour=0, td_idf=None),
 66397: msg(id=66397, type='message', date='2023-10-01T03:38:19', from_name='Дикие Ягодки', from_id='user972724161', reply_to_message_id=None, text='Купила новую камеру.\n1.  трассер \n2. Веб глазок (новая)', words=None, year=2023, month=10, day=1, week=39, weekday=7, hour=3, td_idf=None),
 66398: msg(id=66398, type='message', date='2023-10-01T03:38:19', from_name='Дикие Ягодки', from_id='user972724161', reply_to_message_id=None, text='', words=None, year=2023, month=10, day=1, week=39, weekday=7, hour=3, td_idf=None),
 66399: msg(id=66399, type='message', date='2023-10-01T03:41:18', from_name='Дикие Ягодки', from_id='user972724161', reply_to_message_id=None, text='Извис', words=None, year=2023, month=10, day=1, week=39, weekday

In [4]:
# data[522845].text
# text = data[522835].text


docs = list()

for dp in data:
    text = data[dp].text
    text = emoji.replace_emoji(text, lambda a,_:f".{a}. ")
    text = text.replace(') ', "). ")
    text = text.replace('\n', "\n. ")
    text = '. '.join([a.capitalize() for a in text.split('. ')])
    try:
        doc = Doc(text)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        doc.parse_syntax(syntax_parser)
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
        docs.append({'id': dp, 'from_name': data[dp].from_id, 'doc': doc})
    except:
        pass
    




In [2572]:
# text = 'Мне алоэ плоховато распутывает и не очень делал волосы скользкими, хотя обычно у меня такой проблемы нет, в итоге получался очень большой расход'
# text = emoji.replace_emoji(text, lambda a,_:f".{a}. ")
# text = text.replace(') ', "). ")
# text = text.replace('\n', "\n. ")
# text = '. '.join([a.capitalize() for a in text.split('. ')])
# doc = Doc(text)
# doc.segment(segmenter)
# doc.tag_morph(morph_tagger)
# doc.parse_syntax(syntax_parser)
# for token in doc.tokens:
#     token.lemmatize(morph_vocab)
# doc.tokens

In [546]:

# token_index = 0

from collections import Counter

context_len = 1
words = dict()
words_count = Counter()

for doc_index, doc in enumerate(docs):
    for sent_index, sent in enumerate(doc['doc'].sents):
        for token_index, token in enumerate(sent.tokens):
            if token_index < context_len:
                left_context = 0
            else:
                left_context = token_index-context_len
            right_context = token_index+context_len+1

            tokens = sent.tokens[left_context : token_index] + sent.tokens[token_index+1 : right_context]
            context_lemmas = [l.lemma for l in tokens]
            target_lemma = token.lemma
            # print(token.lemma, context_lemmas)
            if target_lemma not in words:
                words[target_lemma] = Counter()
            words[target_lemma].update(context_lemmas)
            words_count[target_lemma] += 1

# words

words_count = pd.Series(words_count)

In [548]:
words_count = words_count[words_count > 10]
words_count.sort_values(ascending=False).head(40)
words_count

уже             875
объяснить        30
и              5599
это            2125
печальный        12
               ... 
валентинка       12
🔵                22
симка            11
загрязнение      16
удаленка         12
Length: 1933, dtype: int64

In [549]:
df = pd.DataFrame(words)


In [550]:
df = df.loc[words_count.index, words_count.index]

In [551]:
df

,уже,объяснить,и,это,печальный,купить,новый,камера,.,1,...,пробить,фотография,📌,исторический,🙆🏼‍♀️,валентинка,🔵,симка,загрязнение,удаленка
уже,NaN,1.0,38.0,24.0,NaN,NaN,2.0,1.0,25.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
объяснить,1.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
и,38.0,4.0,2.0,71.0,NaN,6.0,9.0,20.0,21.0,17.0,...,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
это,24.0,NaN,71.0,4.0,1.0,1.0,2.0,1.0,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
печальный,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
валентинка,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
🔵,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
симка,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
загрязнение,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [552]:
a = df.sum(axis=1)
similarity_matrix = df/a

similarity_matrix = similarity_matrix.fillna(0).transpose()
similarity_matrix = similarity_matrix[similarity_matrix.sum(axis=1) > 0]


In [553]:
df.count(axis=1).sort_values(ascending=False).head(20)

,       1654
.       1526
и       1310
не      1143
в       1090
?       1050
на      1020
быть     908
с        874
я        757
весь     728
что      721
)        683
по       656
это      623
они      603
вы       600
за       594
как      582
...      566
dtype: int64

In [554]:
import scipy

def dist(a, b):
    return scipy.spatial.distance.cosine(a, b)


In [561]:
target_word = 'подмена'

In [562]:
a = similarity_matrix.loc[target_word]
len(a)


1933

In [563]:
distance1 = dict()

for word, *b in similarity_matrix.itertuples():
    distance1[word] = dist(a, b)
    # break



In [564]:
similarity_matrix.loc[target_word][similarity_matrix.loc[target_word] > 0].sort_values(ascending=False)

,         0.118710
и         0.054194
.         0.049032
за        0.042581
?         0.040000
            ...   
логика    0.001290
клиент    0.001290
выдача    0.001290
два       0.001290
лететь    0.001290
Name: подмена, Length: 208, dtype: float64

In [565]:
# similarity_matrix.loc['терпение'][similarity_matrix.loc['терпение'] > 0].sort_values(ascending=False)

In [566]:
pd.Series(distance1).sort_values().head(20)

подмена       0.000000
товар         0.116505
брак          0.135800
недостача     0.163910
пакет         0.166069
клиент        0.172405
прийти        0.177563
человек       0.180763
посмотреть    0.182251
хороший       0.182373
говорить      0.182415
написать      0.182720
теперь        0.183356
сотрудник     0.184683
да            0.185561
нет           0.185911
смотреть      0.190941
население     0.191666
писать        0.195272
рейтинг       0.201631
dtype: float64